# WSI - Ćwiczenie 7

*Autor: Maksymilian Nowak*

### Cel ćwiczenia

Celem ćwiczenia jest implementacja narzędzia do wnioskowania przy użyciu sieci Bayesa i zademonstrowanie jego możliwości na następującym przykładzie:
- Mamy w domu zamontowany alarm przeciwwłamaniowy.
- Czasem alarm uruchamia się w przypadku niewielkich trzęsień ziemi.
- Umówiliśmy się z sąsiadami, aby dzwonili gdy usłyszą alarm.
    - Jan dzwoni do nas zawsze gdy usłyszy sygnał alarmu - czasem myli go nawet z innymi dźwiękami.
    - Magda dzwoni rzadziej - w domu lubi słuchać dość głośnej muzyki.
    
**Pytanie: Jak ocenimy szanse na to, że było włamanie w zależności od tego kto zadzwonił?**

#### Założenia

1. Wnioskowanie będzie stosowało algorytm MCMC z próbkowaniem Gibbsa,
2. Metoda/funkcja do wnioskowania będzie mieć 3 parametry wejściowe:
    1. Dowody - zaobserwowane wartości wybranych węzłów sieci,
    2. Zapytanie - określenie, dla której zmiennej chcemy wykonać obliczenia,
    3. Liczba iteracji algorytmu MCMC,
    
    oraz będzie zwracała zaktualizowaną tabelę prawdopodobieństw dla zmiennej z zapytania,
3. Można przyjąć założenie, że wszystkie zmienne losowe są binarne,
4. Eksperymenty powinny dotyczyć tego, jak zmieniają się wyniki wnioskowania i czas obliczeń wraz ze zwiększaniem liczby iteracji.


In [51]:
nodes = {
    'Burglary': [],
    'Earthquake': [],
    'Alarm': ['Burglary', 'Earthquake'],
    'JanCalls': ['Alarm'],
    'MagdaCalls': ['Alarm']
}

probabilities = {
    'Burglary': {
        True: 0.01,
        False: 0.99
    },
    'Earthquake': {
        True: 0.02,
        False: 0.98
    },
    'Alarm': {
        True: {
            (True, True): 0.95,
            (True, False): 0.94,
            (False, True): 0.29,
            (False, False): 0.001
        },
        False: {
            (True, True): 0.05,
            (True, False): 0.06,
            (False, True): 0.71,
            (False, False): 0.999
        }
    },
    'JanCalls': {
        True: {
            (True,): 0.90,
            (False,): 0.05
        },
        False: {
            (True,): 0.10,
            (False,): 0.95
        }
    },
    'MagdaCalls': {
        True: {
            (True,): 0.70,
            (False,): 0.01
        },
        False: {
            (True,): 0.30,
            (False,): 0.99
        }
    }
}

In [52]:
class BayesianNode:
    def __init__(self, name):
        self.name = name
        self.parents = []
        self.children = []
        self.value = None
    
    def __str__(self):
        return self.name
    
    def add_parent(self, parent):
        self.parents.append(parent)
    
    def add_child(self, child):
        self.children.append(child)
    
    def set_value(self, value):
        self.value = value

In [53]:
class BayesianNetwork:
    def __init__(self, nodes_struct, probabilities):
        self.nodes = []
        self.probabilities = probabilities
        self.build(nodes_struct)
    
    def build(self, nodes_struct):
        for node_name in nodes_struct:
            node = BayesianNode(node_name)
            self.nodes.append(node)
        
        for node_name in nodes_struct:
            node = self.get_node(node_name)
            for parent_name in nodes_struct[node_name]:
                parent = self.get_node(parent_name)
                node.add_parent(parent)
                parent.add_child(node)
    
    def get_node(self, node_name):
        for node in self.nodes:
            if node.name == node_name:
                return node
        return None
    
    def conditional_prob(self, node_name, value, parent_values=None):
        node = self.get_node(node_name)
        prob_table = self.probabilities[node.name]
        if len(node.parents) == 0:
            return prob_table[value]
        else:
            return prob_table[value][parent_values]
    
    def markov_blanket_prob(self, node_name, value):
        node = self.get_node(node_name)

        # P(node | parents)
        if len(node.parents) > 0:
            prob = self.conditional_prob(node_name, value, tuple([parent.value for parent in node.parents]))
        else:
            prob = self.conditional_prob(node_name, value)
        # Get children
        children = node.children
        # P(children | children_parents)
        for child in children:
            prob *= self.conditional_prob(child.name, child.value, tuple([child_parent.value for child_parent in child.parents]))
        # P(parents | parents_children)
        # for parent in node.parents:
        #     prob *= self.conditional_prob(parent.name, parent.value, tuple([parent_child.value for parent_child in parent.children]))
        return prob

In [54]:
import numpy as np

class Gibbs:
    def __init__(self, network):
        self.network = network
        self.counts = {}
    
    def run(self, evidence, query, iterations):
        for node in self.network.nodes:
            if node.name in evidence:
                node.set_value(evidence[node.name])
            else:
                node.set_value(np.random.choice([True, False]))
        self.counts = {True: 0, False: 0}
        for _ in range(iterations):
            not_evidence = [node for node in self.network.nodes if node.name not in evidence]
            node = np.random.choice(not_evidence)
            blanket_prob_true = self.network.markov_blanket_prob(node.name, True)
            blanket_prob_false = self.network.markov_blanket_prob(node.name, False)
            prob_true = blanket_prob_true / (blanket_prob_true + blanket_prob_false)
            prob_false = blanket_prob_false / (blanket_prob_true + blanket_prob_false)
            node.set_value(np.random.choice([True, False], p=[prob_true, prob_false]))
            query_node = self.network.get_node(query)
            self.counts[query_node.value] += 1
        return {True: self.counts[True] / iterations, False: self.counts[False] / iterations}

In [55]:
bn = BayesianNetwork(nodes, probabilities)
gibbs = Gibbs(bn)
print(gibbs.run({'Burglary': True, 'Earthquake': False}, 'JanCalls', 10000))

{True: 0.8561, False: 0.1439}
